In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
white_path = "wine-white.csv"
red_path = "wine-red.csv"

def import_merge_df(file_name1, file_name2):
    df1 = pd.read_csv(file_name1)
    df2 = pd.read_csv(file_name2)
    print(df1.shape)
    print(df2.shape)
    # merge the dataframes 
    df = pd.concat([df1, df2], ignore_index=True)
    return df


df = import_merge_df(red_path, white_path)
df

(1599, 12)
(4898, 12)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


### Setup data

In [56]:
# generate features and labels
features = df.drop('quality', 1)
labels = df['quality']
print(labels.name+ ' mean')

# split the data into train and test

# first find the average quality rating
print(labels.mean())

# optimize seeds so the test and train data has similar mean quality
def test_seeds():
    test_prop = 0.25
#     seed = 23
    seeds = range(3000, 3006)
    for seed in seeds:
        X_train, X_test, y_train, y_test = train_test_split(features, labels, 
                                                        test_size = test_prop, random_state = seed)
        print(seed)
        print(y_train.mean())
        print(y_test.mean())
    
test_seeds()

quality mean
5.818377712790519
3000
5.825123152709359
5.798153846153846
3001
5.818349753694581
5.818461538461539
3002
5.817118226600985
5.822153846153846
3003
5.821223316912972
5.809846153846154
3004
5.817528735632184
5.820923076923077
3005
5.814449917898194
5.830153846153846


In [57]:
# use 3001 as the seed

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.25, 
                                                    random_state = 3001)


## Linear Regression

In [58]:
lr = LinearRegression().fit(X_train, y_train)
print(lr.coef_)
print("Training set score: %f" % lr.score(X_train, y_train))
print("Testing set score: %f" % lr.score(X_test, y_test))

[ 6.73681277e-02 -1.19427983e+00 -4.06444344e-02  4.39575763e-02
 -6.28414453e-01  6.26955856e-03 -2.25175604e-03 -5.82380039e+01
  4.37530069e-01  8.21094675e-01  2.68498290e-01]
Training set score: 0.297718
Testing set score: 0.273965


## Ridge

In [54]:
ridge = Ridge().fit(X_train, y_train)
print("Training set score: %f" % ridge.score(X_train, y_train))
print("Testing set score: %f" % ridge.score(X_test, y_test))

Training set score: 0.359441
Testing set score: 0.352074


## Lasso

In [59]:
lasso = Lasso().fit(X_train, y_train)
print("Training set score: %f" % ridge.score(X_train, y_train))
print("Testing set score: %f" % ridge.score(X_test, y_test))

Training set score: 0.227666
Testing set score: 0.221048
